In [27]:
class InputNode(Node): # placeholder node
    def __init__(self, name=None):
        super().__init__(name)
        self.value = None


class LinearNode(Node):
    def __init__(self, parent, weights, bias, name=None):
        super().__init__(name)
        self.parents = [parent]
        parent.children.append(self)
        self.weights = weights
        self.bias = bias
        self.value = None

    def forward(self):
        x = self.parents[0].value
        if x is None:
            raise ValueError(f"Parent value of {self} is not set.")
        out_dim = len(self.bias)
        in_dim = len(x)
        result = []
        for i in range(out_dim):
            s = self.bias[i]
            for j in range(in_dim):
                s += self.weights[i][j] * x[j]
            result.append(s)
        self.value = result


class ActivationNode(Node):
    def __init__(self, parent, name=None):
        super().__init__(name)
        self.parents = [parent]
        parent.children.append(self)
        self.value = None

    def _activate(self, x_element):
        raise NotImplementedError("Override in subclass.")

    def forward(self):
        x = self.parents[0].value
        if x is None:
            raise ValueError(f"Parent value of {self} is not set.")
        self.value = [self._activate(xi) for xi in x]


class ReLUNode(ActivationNode):
    def _activate(self, x_element):
        return x_element if x_element > 0 else 0


class SigmoidNode(ActivationNode):
    def _activate(self, x_element):
        import math
        return 1 / (1 + math.exp(-x_element))


class TanhNode(ActivationNode):
    def _activate(self, x_element):
        import math
        return math.tanh(x_element)


In [28]:
def compute_outputs(graph, feed_dict): #Perform a forward pass on a feed‐forward DAG.
    topo_list = graph.topological_sort()
    for node in topo_list:
        if node in feed_dict:
            node.value = feed_dict[node]
        else:
            node.forward()

    # Return a mapping node
    return {node: node.value for node in topo_list}


In [29]:
def compute_error(graph, root, feed_dict, true_label): # Compute the squared‐error
    # 1) Forward pass to populate every node.value
    values = compute_outputs(graph, feed_dict)

    # 2) Extract the predicted vector from the root node
    if root not in values:
        raise KeyError("The specified root node is not in the graph.")
    prediction = values[root]

    # 3) Ensure prediction and true_label have the same length
    if len(prediction) != len(true_label):
        raise ValueError("Length mismatch between prediction and true_label.")

    # 4) Compute sum of squared errors
    error = 0.0
    for p, t in zip(prediction, true_label):
        diff = p - t
        error += diff * diff

    return error


In [34]:
from keras.datasets import boston_housing
import random

# load data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()


x_sample = x_train[0]
y_true = y_train[0]

# Convert the feature vector into a Python list
input_vector = x_sample.tolist()
true_label    = [y_true]

# Randomly initialize a 1×13 weight matrix and a bias of length 1
W = [[random.uniform(-1, 1) for _ in range(13)]]
b = [random.uniform(-1, 1)]

# Create the nodes
x   = InputNode()
lin = LinearNode(x, W, b)

# Build graph
G = ComputationGraph()
G.add_edge(x, lin)

# 4) Run the forward pass on that single example
feed_dict = { x: input_vector }
values   = compute_outputs(G, feed_dict)
y_pred   = lin.value    # a list of length 1 (e.g. [some_float])

# 5) Compute squared-error loss against the true label
error = compute_error(G, lin, feed_dict, true_label)

# 6) Print everything out
print("Input features (13 dims):", input_vector)
print("True target (one value):", true_label)
print("Predicted (linear) output:", y_pred)
print("Squared‐error loss:", error)


Input features (13 dims): [1.23247, 0.0, 8.14, 0.0, 0.538, 6.142, 91.7, 3.9769, 4.0, 307.0, 21.0, 396.9, 18.72]
True target (one value): [np.float64(15.2)]
Predicted (linear) output: [3.024064218940629]
Squared‐error loss: 148.25341214448187
